<a href="https://colab.research.google.com/github/CalculatedContent/ww-phys_theory/blob/master/Compare_Jacobian_Diag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Compare Diagonal Jacobian on ResNet 1 batch

Why is my data different from Johns ?


In [1]:
!pip install torch==1.5.0+cu101 torchvision==0.6.0+cu101 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html


In [0]:
thismodel = 'resnet20_cifar10'
inum_start = 0

In [3]:
!pip install ipython-autotime
%load_ext autotime

  Created wheel for ipython-autotime: filename=ipython_autotime-0.1-cp36-none-any.whl size=1832 sha256=b634cf0690bcb76d2f6ba149c080964ce764b231a60bb1af48ac11686cd22bd4
  Stored in directory: /root/.cache/pip/wheels/d2/df/81/2db1e54bc91002cec40334629bc39cfa86dff540b304ebcd6e
Successfully built ipython-autotime


In [4]:
!pip install pytorchcv

     |████████████████████████████████| 440kB 2.7MB/s 
time: 4.69 s


In [5]:
!pip install GPUtil
import GPUtil

GPUtil.showUtilization()

  Created wheel for GPUtil: filename=GPUtil-1.4.0-cp36-none-any.whl size=7413 sha256=94e987022a6e2dddbfd9e597112a2dd575f5b9f397c89fc905b101d1334619fa
  Stored in directory: /root/.cache/pip/wheels/3d/77/07/80562de4bb0786e5ea186911a2c831fdd0018bda69beab71fd
Successfully built GPUtil
| ID | GPU | MEM |
------------------
|  0 |  0% |  0% |
time: 4.63 s


In [6]:
!import os
import time

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.data.dataset import Subset

from torchvision import datasets
from torchvision import transforms

import matplotlib.pyplot as plt
from PIL import Image

import math
import copy


/bin/bash: import: command not found
time: 5.83 s


In [7]:
def get_data(batch_size=100, train_range=None, test_range=None):
  normalize = transforms.Normalize(mean=[x/255.0 for x in [125.3, 123.0, 113.9]], std=[x / 255.0 for x in [63.0, 62.1, 66.7]])
  transform_train = transforms.Compose([
          transforms.ToTensor(),
          normalize])
  transform_test = transforms.Compose([
        transforms.ToTensor(),
        normalize])
  train_dataset = datasets.CIFAR10(root='data', 
                                train=True, 
                                transform=transform_train,
                                download=True)
  test_dataset = datasets.CIFAR10(root='data', 
                                train=False, 
                                transform=transform_test,
                                download=True)  
  
  if train_range:
      train_dataset = torch.utils.data.Subset(train_dataset, train_range)

  if test_range:
      teat_dataset = torch.utils.data.Subset(test_dataset, test_range)


  train_loader = DataLoader(dataset=train_dataset, 
                          batch_size=batch_size,
                          num_workers=4,
                          shuffle=False)
  test_loader = DataLoader(dataset=test_dataset, 
                         batch_size=batch_size,
                         num_workers=4,
                         shuffle=False)
  return train_dataset, test_dataset, train_loader, test_loader

time: 10.6 ms


### $J$ Jacobian (replaces native pytorch 1.5 code)

Uses autograd 





In [8]:
from torch.autograd.gradcheck import zero_gradients

def fast_jacobian(inputs, output, batch_size, data_dim):
	"""
	input: input for the function for which the Jacobian will
	computed. It will be batch_size*data_dim. Make sure that the
	input is flagged as requires_grad=True with the torch.autograd.Variable
	wrapper. 
	output: output of the function for which the Jacobian will
	be computed. It will be batch_size*classes
	return: Jacobian of dimension batch_size*classes*data_dim
	"""
	assert inputs.requires_grad
	print("inputs: ",inputs.shape)
	num_classes = output.size()[1] #0 index is batch

	jacobian = torch.zeros(num_classes, *inputs.size())
	grad_output = torch.zeros(*output.size())
	if inputs.is_cuda:
		grad_output = grad_output.cuda()
		jacobian = jacobian.cuda()

	#zero out gradients
	#compute gradient for one of the classes
	for i in range(num_classes):
		zero_gradients(inputs)
		grad_output.zero_()
		grad_output[:,i] = 1
		output.backward(grad_output, retain_graph=True)
		jacobian[i] = inputs.grad.data

	J =  torch.transpose(jacobian, dim0=0, dim1=1)
	print("fast J: ",J.shape, data_dim)
	J = J.reshape(batch_size, data_dim)
 
	return J


time: 10.9 ms


### Compare old and new diagonal;


### ResNet 

In [9]:
import pytorchcv
from pytorchcv.model_provider import get_model as ptcv_get_model

for modelname in pytorchcv.model_provider._models.keys():
    if modelname.startswith('resnet') and modelname.endswith('cifar10'):
        print(modelname)

resnet20_cifar10
resnet56_cifar10
resnet110_cifar10
resnet164bn_cifar10
resnet272bn_cifar10
resnet542bn_cifar10
resnet1001_cifar10
resnet1202_cifar10
time: 99.5 ms


In [14]:
num_classes= 10
device = 'cuda:0'

data_dim = 3*32*32

#model = ptcv_get_model(thismodel, pretrained=True)
model = getMLP()
model = model.to(device)

for batch_size in range(1,5):
  print("batch size ", batch_size)
  train_dataset, test_dataset, train_loader, test_loader = get_data(batch_size=batch_size)

  for batch, data in enumerate(test_loader):
    features, labels = data
    print(labels)

    f0 = features[0,:,:,:]
    print("||x[0]|| ",np.linalg.norm(f0.numpy()))
    features[1:,:,:,:]=0.0

    inputs = features.to(device)
    inputs.requires_grad=True
    outputs = model(inputs)

    Jnew = fast_jacobian(inputs, outputs, batch_size, num_classes*data_dim)
    j0new = Jnew.cpu().numpy()[0,:]

    Jold = torch.autograd.functional.jacobian(model, inputs)
    Jold = Jold.transpose(1,2)
    print("Jold: ",Jold.shape)

    il = labels[0]
    print(il)
    j0orig = Jold.cpu().numpy()[0,0,:,:,:,:]
    j0old = Jold.cpu().numpy()[0,:,:,:,:,:]

    print("||jnew|| ", np.linalg.norm(j0new))
    print("||jorig|| ", np.linalg.norm(j0orig))
    print("||jold|| ", np.linalg.norm(j0old))
    print(".   ")

    break



batch size  1
Files already downloaded and verified
Files already downloaded and verified
tensor([3])
||x[0]||  41.729523
inputs:  torch.Size([1, 3, 32, 32])
fast J:  torch.Size([1, 10, 3, 32, 32]) 30720
Jold:  torch.Size([1, 1, 10, 3, 32, 32])
tensor(3)
||jnew||  4.094996
||jorig||  4.094996
||jold||  4.094996
.   
batch size  2
Files already downloaded and verified
Files already downloaded and verified
tensor([3, 8])
||x[0]||  41.729523
inputs:  torch.Size([2, 3, 32, 32])
fast J:  torch.Size([2, 10, 3, 32, 32]) 30720
Jold:  torch.Size([2, 2, 10, 3, 32, 32])
tensor(3)
||jnew||  4.094996
||jorig||  4.094996
||jold||  4.094996
.   
batch size  3
Files already downloaded and verified
Files already downloaded and verified
tensor([3, 8, 8])
||x[0]||  41.729523
inputs:  torch.Size([3, 3, 32, 32])
fast J:  torch.Size([3, 10, 3, 32, 32]) 30720
Jold:  torch.Size([3, 3, 10, 3, 32, 32])
tensor(3)
||jnew||  4.094996
||jorig||  4.094996
||jold||  4.094996
.   
batch size  4
Files already downloade


 Model Jacobian must depend on batch size ?

 OR Is it the Batch Normalization

 TRY a 1 layer MLP


In [11]:
class MLP(nn.Module):
  def __init__(self, n_units, init_scale=1.0):
    super(MLP, self).__init__()

    self._n_units = copy.copy(n_units)
    self._layers = []
    for i in range(1, len(n_units)):
      layer = nn.Linear(n_units[i-1], n_units[i], bias=False)
      variance = math.sqrt(2.0 / (n_units[i-1] + n_units[i]))
      layer.weight.data.normal_(0.0, init_scale * variance)
      self._layers.append(layer)

      name = 'fc%d' % i
      if i == len(n_units) - 1:
        name = 'fc'  # the prediction layer is just called fc
      self.add_module(name, layer)


  def forward(self, x):
    x = x.view(-1, self._n_units[0])
    out = self._layers[0](x)
    for layer in self._layers[1:]:
      out = F.relu(out)
      out = layer(out)
    return out

time: 12.3 ms


In [13]:
def getMLP():
  mlp_spec = '512'
  n_units = [int(x) for x in mlp_spec.split('x')] # hidden dims
  n_units.append(10)  # output dim
  n_units.insert(0, 32*32*3)        # input dim
  model = MLP(n_units)
  return model


time: 2.37 ms
